In [3]:
import tensorflow as tf
x_data =[[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
#One-hot encoding으로 표시함 / 2 2 2 1 1 1 0 0(몇 번째 자리가 1인지 - 0부터 시작)
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]] 

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3]) 
nb_classes = 3 #One-hot 사용할 때는 y의 개수가 레이블의 개수가 된다

#들어오는 값(4개), 나가는 값(3개) -shape 주의
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

#텐서플로우의 softmax 라이브러리 이용하기
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

#Gradient Descent 사용
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

#세션
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
            _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})

            if step % 200 == 0:
                print(step, cost_val)

   
    #tensorflow의 argmax -> 몇 번째 있는 것이 가장 높은 값을 가지는지 반환해줌(0,1,2)
    all = sess.run(hypothesis,feed_dict={X:[[1,11,7,9],[1,3,4,3],[1,1,0,1]]})
    print(all, sess.run(tf.argmax(all,1)))

0 5.012436
200 0.5726346
400 0.4673478
600 0.39174896
800 0.3217855
1000 0.25085443
1200 0.22029363
1400 0.20085068
1600 0.18444283
1800 0.17042184
2000 0.15831138
[[8.8343788e-03 9.9115491e-01 1.0743642e-05]
 [8.1473106e-01 1.6261466e-01 2.2654226e-02]
 [1.6100721e-08 3.6520994e-04 9.9963474e-01]] [1 0 2]


In [5]:
import tensorflow as tf
import numpy as np
#데이터 읽어오기
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  #0~6 1개씩, shape(?,1)

#One-hot 만들기
#one-hot에서는 총 클래스의 수를 매개변수로 넘겨주는데, 차원이 1 증가하게 됨
#(Rank = N, one_hot의 아웃풋은 N+1)
#ex. [[0],[3]] -> [[[1,0,0,0,0,0,0], [0,0,0,1,0,0,0]]] 이런 식으로 2차원 -> 3차원으로 증가함
Y_one_hot = tf.one_hot(Y, nb_classes)  #one hot shape(?,1,7)
print("one_hot:", Y_one_hot)
#해결하기 위해 'reshape'를 사용함
#[-1,7]을 매개변수로 넣으면 [[1,0,0,0,0,0,0],[0,0,0,1,0,0,0]]이 나온다.
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) #shape = (?,7)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight') #입력 16개, 출력 7개
b = tf.Variable(tf.random_normal([nb_classes]), name='bias') #출력 7개

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

#cost 계산
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1) #0~6사이 중 하나의 값으로 만드는 과정
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #예측과 실제값(Y) 같을까?
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 세션
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))
    #잘 예측했는지를 출력
    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()): #flatten [[1]. [0]] -> [1,0] 이런 식으로 만들어 주는 것
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


(101, 16) (101, 1)
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step:     0	Cost: 7.588	Acc: 19.80%
Step:   100	Cost: 0.688	Acc: 82.18%
Step:   200	Cost: 0.436	Acc: 88.12%
Step:   300	Cost: 0.319	Acc: 90.10%
Step:   400	Cost: 0.250	Acc: 96.04%
Step:   500	Cost: 0.204	Acc: 96.04%
Step:   600	Cost: 0.171	Acc: 96.04%
Step:   700	Cost: 0.147	Acc: 98.02%
Step:   800	Cost: 0.128	Acc: 98.02%
Step:   900	Cost: 0.114	Acc: 98.02%
Step:  1000	Cost: 0.103	Acc: 99.01%
Step:  1100	Cost: 0.093	Acc: 99.01%
Step:  1200	Cost: 0.085	Acc: 99.01%
Step:  1300	Cost: 0.079	Acc: 100.00%
Step:  1400	Cost: 0.073	Acc: 100.00%
Step:  1500	Cost: 0.069	Acc: 100.00%
Step:  1600	Cost: 0.064	Acc: 100.00%
Step:  1700	Cost: 0.061	Acc: 100.00%
Step:  1800	Cost: 0.058	Acc: 100.00%
Step:  1900	Cost: 0.055	Acc: 100.00%
St